In [1]:
import os
import sys
import re

import numpy as np
import pandas as pd
import scipy as sp
from scipy import signal

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sys.path.insert(0, '../src')
from features import *
from utils import *

In [3]:
sns.set(rc={'figure.figsize':(20,5)})

In [4]:
mbit_rate = 1/125000

low_fp = '../data/240p/' 
threesixty_fp = '../data/360p/' 
med_fp = '../data/480p/'
seventwenty_fp = '../data/720p/' 
high_fp = '../data/1080p/' 

In [17]:
low_dfs = []
for file in os.listdir(low_fp):
    if file != '.ipynb_checkpoints' and file != '.DS_Store':
        low_dfs.append(pd.read_csv(low_fp+file))
    
threesixty_dfs = []
for file in os.listdir(threesixty_fp):
    if file != '.ipynb_checkpoints' and file != '.DS_Store':
        threesixty_dfs.append(pd.read_csv(threesixty_fp+file))
        
med_dfs = []
for file in os.listdir(med_fp):
    if file != '.ipynb_checkpoints' and file != '.DS_Store':
        med_dfs.append(pd.read_csv(med_fp+file))
        
seventwenty_dfs = []
for file in os.listdir(seventwenty_fp):
    if file != '.ipynb_checkpoints' and file != '.DS_Store':
        seventwenty_dfs.append(pd.read_csv(seventwenty_fp+file))
    
high_dfs = []
for file in os.listdir(high_fp):
    if file != '.ipynb_checkpoints' and file != '.DS_Store':
        high_dfs.append(pd.read_csv(high_fp+file))

In [18]:
# stdoan_low = pd.read_csv(low_fp + 'stdoan-101-action-240p-20201127.csv')
# stdoan_med = pd.read_csv(med_fp + 'stdoan-101-action-480p-20201127.csv')
# stdoan_high = pd.read_csv(high_fp + 'stdoan-101-action-1080p-20201127.csv')

In [20]:
low_ms = []
for df in low_dfs:
    low_ms.append(convert_ms_df(df,True))
    
threesixty_ms = []
for df in threesixty_dfs:
    threesixty_ms.append(convert_ms_df(df,True))
    
med_ms = []
for df in med_dfs:
    med_ms.append(convert_ms_df(df,True))
    
seventwenty_ms = []
for df in seventwenty_dfs:
    seventwenty_ms.append(convert_ms_df(df,True))
    
high_ms = []
for df in high_dfs:
    high_ms.append(convert_ms_df(df,True))

In [ ]:
# low_ms = convert_ms_df(stdoan_low, True)
# med_ms = convert_ms_df(stdoan_med, True)
# high_ms = convert_ms_df(stdoan_high, True)

In [21]:
low_resamples = []
for df in low_ms:
    low_resamples.append(df.resample('500ms', on='Time').sum())
    
threesixty_resamples = []
for df in threesixty_ms:
    threesixty_resamples.append(df.resample('500ms', on='Time').sum())
    
med_resamples = []
for df in med_ms:
    med_resamples.append(df.resample('500ms', on='Time').sum())
    
seventwenty_resamples = []
for df in seventwenty_ms:
    seventwenty_resamples.append(df.resample('500ms', on='Time').sum())
    
high_resamples = []
for df in high_ms:
    high_resamples.append(df.resample('500ms', on='Time').sum())

In [22]:
# low_resample = low_ms.resample('500ms', on='Time').sum()
# med_resample = med_ms.resample('500ms', on='Time').sum()
# high_resample = high_ms.resample('500ms', on='Time').sum()


## Aggregate Features

In [23]:
## take the aggregate features of the whole chunk; download and upload
def agg_feat(df, col):
    return [np.mean(df[col]), np.std(df[col])]

## take the ratio of upload:download packets
def pkt_ratio(df):
    ms_df = convert_ms_df(df, True)
    local = np.sum(ms_df['pkt_src'] == '1') 
    server = np.sum(ms_df['pkt_src'] == '2') 
    return local / server

## take the ratio of upload:download bytes
def bytes_ratio(df):
    local = df['1->2Bytes'].sum()
    server = df['2->1Bytes'].sum()
    return local / server

## Peak Related Aggregate Features

In [24]:
## finds the peaks with mean + 2(1) std
## run the above aggregate functions on the peaks only??

def get_peak_loc(df, col, invert=False):
  'invert arg allows you to get values not considered peaks'
  df_avg = df[col].mean()
  df_std = df[col].std()
  
  threshold = df_avg + (1 * df_std)
  
  if invert:
    return np.array(df[col] < threshold)
  
  else:
    return np.array(df[col] > threshold)

## np.mean, np.var, np.std - think of more?  
def peak_time_diff(df, col):
  '''
  mess around with the different inputs for function. 
  variance seems to inflate the difference betweent the two the most with litte
  to no data manipulation. however, currently trying things like
  squaring the data before taking the aggregate function to exaggerate
  differences (moderate success??)
  '''
  peaks = df[get_peak_loc(df, col)]
  peaks['Time'] = peaks['Time'] - peaks['Time'].min()
  time_diff = np.diff(peaks['Time'] ** 2)
  return [np.mean(time_diff), np.std(time_diff)]

In [25]:
from scipy.signal import find_peaks

def peak_times(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    if list(peaks) == []:
        return [-1]
    times = df.iloc[peaks]['Time'].values
    time_between_peaks = [times[i]-times[i-1]for i in range(1,len(times))]
    #print(time_between_peaks)
    #time_between_peaks[0]=0
    if time_between_peaks == []:
        return -1
    return time_between_peaks

def num_peaks(df,col,thresh):
    x = df[col]
    peaks, _ = find_peaks(x, height=thresh)
    return len(peaks)

## Spectral Features

In [26]:
def spectral_features(df, col):

    """
    welch implemention of spectral features
    resample the data before inputting (might change prereq depending on
    resource allocation)
    """

    f, Pxx_den = sp.signal.welch(df[col], fs=2)
    Pxx_den = np.sqrt(Pxx_den)

    peaks = sp.signal.find_peaks(Pxx_den)[0]
    prominences = sp.signal.peak_prominences(Pxx_den, peaks)[0]

    idx_max = prominences.argmax()
    loc_max = peaks[idx_max]

    return [f[loc_max], Pxx_den[loc_max], prominences[idx_max]]

## Chunking & Feature creation

In [27]:
## wip; need to decide chunk size eventually
## should we also make this chunking feature be our feature creation?

def chunk_data(df, interval=60):

    """
    takes in a filepath to the data you want to chunk and feature engineer
    chunks our data into a specified time interval
    each chunk is then turned into an observation to be fed into our classifier
    """

    df_list = []
    
    df['Time'] = df['Time'] - df['Time'].min()
    
    total_chunks = np.floor(df['Time'].max() / interval).astype(int)

    for chunk in np.arange(total_chunks):
      
        start = chunk * interval
        end = (chunk+1) * interval

        temp_df = (df[(df['Time'] >= start) & (df['Time'] < end)])
        
        df_list.append(temp_df)
        
    return df_list

In [28]:
def create_features(dfs, interval=60):

  features = [
    'dwl_peak_freq',
    'dwl_peak_prom',
    'dwl_max_psd',
    'dwl_bytes_avg',
    'dwl_bytes_std',
    'dwl_peak_avg',
    'dwl_peak_std',
    'upl_peak_freq',
    'upl_peak_prom',
    'upl_max_psd',
    'upl_bytes_avg',
    'upl_bytes_std',
    'upl_peak_avg',
    'upl_peak_std',
    'dwl_time_peak',#'IMAN_up_time_peak',
      'dwl_num_peak'#,'IMAN_up_num_peak'
  ]  

  vals = []
  for df in dfs:
      df_chunks = chunk_data(df, interval)

      for chunk in df_chunks:

        preproc = convert_ms_df(chunk, True)
        upl_bytes = preproc[preproc['pkt_src'] == '1'].resample('500ms', on='Time').sum()
        dwl_bytes = preproc[preproc['pkt_src'] == '2'].resample('500ms', on='Time').sum()

        ## spectral features
        dwl_spectral = spectral_features(dwl_bytes, 'pkt_size')
        upl_spectral = spectral_features(upl_bytes, 'pkt_size')

        ## aggregate features
        dwl_agg = agg_feat(chunk, '2->1Bytes')
        upl_agg = agg_feat(chunk, '1->2Bytes')

        ## peak features
        dwl_peak = peak_time_diff(chunk, '2->1Bytes')
        upl_peak = peak_time_diff(chunk, '1->2Bytes')
        
        ## iman's time between peak 
        iman_dwn_time_peak = np.mean(peak_times(chunk,'2->1Bytes',1000000))
        #iman_up_time_peak = np.mean(peak_times(chunk,'1->2Bytes',50000))
        
        ## iman's num peak
        iman_dwn_num_peak = num_peaks(chunk,'2->1Bytes',1000000)
        #iman_up_num_peak = num_peaks(chunk,'1->2Bytes',50000)


        
        feat_val = np.hstack((
          dwl_spectral,
          dwl_agg,
          dwl_peak,
          upl_spectral,
          upl_agg,
          upl_peak,
            iman_dwn_time_peak,#iman_up_time_peak,
            iman_dwn_num_peak,#iman_up_num_peak
        ))

        vals.append(feat_val)
    
  return pd.DataFrame(columns=features, data=vals).fillna(0)

## Dev Playground

In [64]:
%%time
low_feat = create_features(low_dfs, 120)
threesixty_feat = create_features(threesixty_dfs, 120)
med_feat = create_features(med_dfs, 120)
seventwenty_feat = create_features(seventwenty_dfs, 120)
high_feat = create_features(high_dfs, 120)

CPU times: user 21.3 s, sys: 220 ms, total: 21.5 s
Wall time: 21.6 s


In [65]:
low_feat['resolution'] = '240p'
threesixty_feat['resolution'] = '360p'
med_feat['resolution'] = '480p'
seventwenty_feat['resolution'] = '720p'
high_feat['resolution'] = '1080p'

In [66]:
training = pd.concat([low_feat, threesixty_feat, med_feat,seventwenty_feat, high_feat]).reset_index(drop=True)

In [67]:
## SELECT SUBSETS OF FEATURES

#training = training[['dwl_bytes_avg','dwl_peak_prom','upl_bytes_std','dwl_bytes_std','upl_peak_std','resolution']]
#training = training[['dwl_bytes_avg','upl_max_psd','dwl_max_psd','upl_peak_prom','dwl_num_peak','dwl_peak_prom','resolution']]
#training = training[['dwl_max_psd','upl_max_psd','dwl_peak_prom','upl_peak_prom','dwl_num_peak','dwl_bytes_avg','upl_bytes_std','upl_bytes_avg','resolution']]


In [68]:
training

,dwl_peak_freq,dwl_peak_prom,dwl_max_psd,dwl_bytes_avg,dwl_bytes_std,dwl_peak_avg,dwl_peak_std,upl_peak_freq,upl_peak_prom,upl_max_psd,upl_bytes_avg,upl_bytes_std,upl_peak_avg,upl_peak_std,dwl_time_peak,dwl_num_peak,resolution
0,0.201681,3.172323e+05,3.010390e+05,105818.018349,292073.412690,947.769231,980.253668,0.133891,14302.110672,12720.120720,6901.944954,16862.760631,947.769231,980.253668,13.000000,2.0,240p
1,0.200837,2.116595e+05,2.023631e+05,56392.975904,181129.552286,2242.666667,1420.062049,0.203390,12269.692224,11698.225453,3695.939759,9105.882828,2242.666667,1420.062049,-1.000000,0.0,240p
2,0.101266,2.252769e+05,2.128232e+05,51312.250000,203432.423015,2080.800000,2320.069602,0.100840,13340.341481,12745.342358,3723.068182,10219.868095,2080.800000,2320.069602,-1.000000,1.0,240p
3,0.193277,1.933348e+05,1.817832e+05,39377.755319,151199.194042,1656.200000,1367.217378,0.100840,11055.763175,10565.957392,3055.329787,7590.302042,1656.200000,1367.217378,-1.000000,0.0,240p
4,0.125523,1.908308e+05,1.840692e+05,63535.761905,191208.720401,1513.800000,1348.799970,0.100418,12298.882370,11778.581777,4370.000000,9625.049335,1513.800000,1348.799970,-1.000000,0.0,240p
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,0.093617,1.295016e+06,1.287103e+06,233169.513514,569860.008211,936.333333,666.245992,0.093617,61418.518635,60301.660623,12333.486486,27193.069522,936.333333,666.245992,10.600000,11.0,1080p
183,0.096070,1.164929e+06,1.092980e+06,228929.241379,516817.245537,1002.272727,1507.679378,0.096070,55312.103148,52585.487555,11895.758621,24587.033458,1102.500000,1546.366790,10.571429,8.0,1080p
184,0.184874,7.298143e+05,6.369195e+05,203555.510204,546653.912209,921.600000,904.036526,0.184874,35513.168457,32592.826118,10648.867347,26150.031304,921.600000,904.036526,13.333333,7.0,1080p
185,0.184211,9.803294e+05,9.645212e+05,248339.244444,588179.115896,784.083333,749.898155,0.184211,46757.514827,46135.557627,12752.111111,27889.468266,784.083333,749.898155,9.700000,11.0,1080p


In [69]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score,accuracy_score

In [70]:
X, y = training.drop(columns=['resolution']), training['resolution']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 4)

In [71]:
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42)
#classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 42)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [84]:
## RUN ON TEST DATA
classifier.fit(X,y)
low_test = pd.read_csv('../data/test/sgs008-109-action-480p-20210202.csv')
#low_test = pd.read_csv('../data/test/sgs008-109-action-720p-20210213.csv')
low_test = pd.read_csv('../data/test/stdoan-102-action-720p-20201206.csv')


low_feat = create_features([low_test], 120)

test_low = low_feat#[['dwl_max_psd','upl_max_psd','dwl_peak_prom','upl_peak_prom','dwl_num_peak','dwl_bytes_avg','upl_bytes_std','upl_bytes_avg']]
y_pred = classifier.predict(test_low)
y_pred

array(['480p', '720p', '720p', '360p', '720p', '360p', '360p', '360p',
       '720p', '480p'], dtype=object)

In [72]:
## FOR VALIDATION SET

y_pred = classifier.predict(X_test)

In [73]:
(pd.crosstab(y_test, y_pred, rownames=['Actual Group'], colnames=['Predicted Group']))

Predicted Group,1080p,240p,360p,480p,720p
Actual Group,,,,,
1080p,13,0,0,0,0
240p,0,9,0,0,0
360p,0,1,5,0,0
480p,0,0,1,11,0
720p,2,0,0,0,5


In [74]:
f1_score(y_test, y_pred, average=None)

array([0.92857143, 0.94736842, 0.83333333, 0.95652174, 0.83333333])

In [75]:
accuracy_score(y_test, y_pred)

0.9148936170212766

In [76]:
from sklearn.model_selection import cross_val_score
cross_val_score(classifier,X,y,cv=10)

array([0.6       , 0.85      , 0.9       , 0.85      , 0.7       ,
       0.95      , 0.9       , 1.        , 0.93333333, 0.6       ])

In [57]:
# features = ['dwl_bytes_avg','dwl_peak_prom','upl_bytes_std','dwl_bytes_std','upl_peak_std']
# importances = classifier.feature_importances_
# indices = np.argsort(importances)[::-1]
# for i in indices:
#     print(features[i],': ',importances[i])
    

In [58]:
# features = ['dwl_bytes_avg','upl_max_psd','dwl_max_psd','upl_peak_prom','dwl_num_peak','dwl_peak_prom']
# importances = classifier.feature_importances_
# indices = np.argsort(importances)[::-1]
# for i in indices:
#     print(features[i],': ',importances[i])

In [59]:
features = ['dwl_peak_freq','dwl_peak_prom','dwl_max_psd','dwl_bytes_avg','dwl_bytes_std','dwl_peak_avg',
            'dwl_peak_std','upl_peak_freq','upl_peak_prom','upl_max_psd','upl_bytes_avg','upl_bytes_std',
            'upl_peak_avg','upl_peak_std','dwl_time_peak','dwl_num_peak']
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
for i in indices:
    print(features[i],': ',importances[i])
    

dwl_peak_prom :  0.12932674112109813
dwl_max_psd :  0.12317735884094977
dwl_bytes_avg :  0.11604856195717536
dwl_time_peak :  0.09693324838411006
upl_max_psd :  0.09256348375010617
dwl_num_peak :  0.08788893328210072
dwl_bytes_std :  0.07971873844951466
upl_peak_prom :  0.06778216380896335
upl_bytes_avg :  0.055120304930472326
upl_bytes_std :  0.05025839868824128
dwl_peak_avg :  0.026375183218262915
dwl_peak_std :  0.020286999863416814
upl_peak_avg :  0.015660424192193427
upl_peak_std :  0.014268447510740866
upl_peak_freq :  0.01338761263014186
dwl_peak_freq :  0.011203399372512236


In [32]:
# ## testing that feature method functions correctly

# l_start = 0 
# l_end = 60

# test_chunk = stdoan_low.copy()
# test_chunk['Time'] = test_chunk['Time'] - test_chunk['Time'].min()
# low_chunk = stdoan_low[(stdoan_low['Time'] >= 0) & (stdoan_low['Time'] < 60)]

# low_chunk_ms = convert_ms_df(low_chunk, True)

# upl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '1']
# dwl_ms = low_chunk_ms[low_chunk_ms['pkt_src'] == '2']

# dwl_chunk_rs = dwl_ms.resample('500ms', on='Time').sum()

# f_dwl, Pxx_dwl = sp.signal.welch(dwl_chunk_rs['pkt_size'], fs=2)